In [1]:
from datasets import load_dataset
from bach import ROOT_DIR
import pygame
import numpy as np
import pyarrow
import pretty_midi

D:\Dev\repo\bach\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


pygame 2.6.1 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
ds = load_dataset("drengskapur/midi-classical-music",  cache_dir=ROOT_DIR.parent / "data_cache", split="train")

In [7]:
ds[0]["file_name"]

'data/albeniz-aragon_fantasia_op47_part_6.mid'

In [24]:
from huggingface_hub import hf_hub_download

# Define the repository and filename
repo_id = 'drengskapur/midi-classical-music'
filename = 'example.mid'  # Replace with the actual filename you want to download

# Download the file
midi_file_path = hf_hub_download(repo_id=repo_id, filename=ds[4000]["file_name"], repo_type="dataset", cache_dir=ROOT_DIR.parent / "data_cache")
print(f'MIDI file downloaded to: {midi_file_path}')


MIDI file downloaded to: D:\Dev\repo\bach\data_cache\datasets--drengskapur--midi-classical-music\snapshots\29c3079aedf539041b1cfd8ffd00d5d508681f1f\data\unknown_artist-a_h-beethoven_10.mid


In [12]:
midi_data = pretty_midi.PrettyMIDI("D:\Dev\\repo\\bach\data_cache\data\\bacewitz-sonata_no4_for_piano_violin_mov1.mid")

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\claud\AppData\Local\Temp\ipykernel_20084\4058205898.py:1: SyntaxWarning: invalid escape sequence '\D'
  midi_data = pretty_midi.PrettyMIDI("D:\Dev\\repo\\bach\data_cache\data\\bacewitz-sonata_no4_for_piano_violin_mov1.mid")
D:\Dev\repo\bach\env\Lib\site-packages\pretty_midi\pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [13]:
notes = []
for instrument in midi_data.instruments:
        if not instrument.is_drum:  # Exclude drum tracks
            for note in instrument.notes:
                notes.append([note.start, note.pitch, note.velocity])
notes = sorted(notes, key=lambda x: x[0])  # Sort by start time
notes = np.array(notes)

In [14]:
import pretty_midi
import numpy as np

def tensor_to_midi(d, output_midi_path="generated_music.mid", instrument_name="Acoustic Grand Piano"):
    """
    Convert a tensor output from the model into a MIDI file.

    Args:
        tensor (torch.Tensor or np.ndarray): Model output of shape (sequence_length, 3).
        output_midi_path (str): Path to save the MIDI file.
        instrument_name (str): Instrument for the MIDI track.
    """


    # Create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI()

    # Create an instrument (default is piano)
    instrument = pretty_midi.Instrument(program=pretty_midi.instrument_name_to_program(instrument_name))

    for note_data in d:
        start_time, pitch, velocity = note_data

        # Ensure values are within valid ranges
        pitch = int(np.clip(pitch, 0, 127))
        velocity = int(np.clip(velocity, 0, 127))
        duration = 0.4  # Default note duration (can be adjusted dynamically)

        # Create a note and add it to the instrument
        note = pretty_midi.Note(velocity=velocity, pitch=pitch, start=start_time, end=start_time + duration)
        instrument.notes.append(note)

    # Add instrument to MIDI
    midi.instruments.append(instrument)

    # Save MIDI file
    midi.write(output_midi_path)
    print(f"Saved generated MIDI to {output_midi_path}")

tensor_to_midi(notes)

Saved generated MIDI to generated_music.mid


In [6]:
def events_to_tokens(events, quantization_step=0.25):
    token_sequence = []
    prev_time = 0.0
    for event in events:
        delta = event['time'] - prev_time
        num_steps = int(round(delta / quantization_step))
        if num_steps > 0:
            token_sequence.extend(['time_shift'] * num_steps)
        token = f"{event['type']}_{event['pitch']}"
        token_sequence.append(token)
        prev_time = event['time']
    return token_sequence

In [7]:
e = events_to_tokens(events)

In [23]:
notes = {}
for instrument in midi_data.instruments:
                if not instrument.is_drum:
                    notes[instrument.name] = []
                    for note in instrument.notes:
                        notes.append(note.pitch)

notes = np.array(notes)

In [26]:
# Initialize the mixer
pygame.mixer.init()

# Load the MIDI file
pygame.mixer.music.load(midi_file_path)

# Play the MIDI file
pygame.mixer.music.play()

In [ ]:

pygame.mixer.init()
pygame.mixer.music.load(ds[0])

In [15]:
import torch

In [24]:
d = torch.load("D:\\Dev\\repo\\bach\\data_cache\\dataset.pt", weights_only=False)